In [ ]:
from google.colab import drive
import zipfile
import numpy as np
import os
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,  Dropout, Flatten, BatchNormalization, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision, Accuracy, Recall
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers, Model
from tensorflow.keras.models import load_model

from keras.utils import to_categorical
from keras.models import load_model

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with zipfile.ZipFile("/content/drive/MyDrive/cv_data/raw_batik_v2.zip") as zip_ref:
  zip_ref.extractall("./")

# Constants

In [ ]:
img_width = 224
img_height = 224
channels = 3
total_inputs = img_width * img_height * channels

batch_size = 32


# Data Gathering

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

# Set up data generators for training, validation, and testing
train_generator = train_datagen.flow_from_directory(
    directory=os.path.join(DATA_DIR, "train"),
    target_size=(img_width, img_height),
    color_mode='rgb',
    class_mode='categorical'  # For multi classes
)

validation_generator = validation_datagen.flow_from_directory(
    directory=os.path.join(DATA_DIR, "validation"),
    target_size=(img_width, img_height),
    color_mode='rgb',
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    directory=os.path.join(DATA_DIR, "test"),
    target_size=(img_width, img_height),
    color_mode='rgb',
    class_mode='categorical'
)

# Print class labels
print("Class labels:")
print(train_generator.class_indices)
DATA_DIR = "/content/raw_batik_v1"

Found 520 images belonging to 20 classes.
Found 120 images belonging to 20 classes.
Found 160 images belonging to 20 classes.
Class labels:
{'Aceh_Pintu_Aceh': 0, 'Bali_Barong': 1, 'Bali_Merak': 2, 'DKI_Ondel_Ondel': 3, 'JawaBarat_Megamendung': 4, 'JawaTimur_Pring': 5, 'Kalimantan_Dayak': 6, 'Lampung_Gajah': 7, 'Madura_Mataketeran': 8, 'Maluku_Pala': 9, 'NTB_Lumbung': 10, 'Papua_Asmat': 11, 'Papua_Cendrawasih': 12, 'Papua_Tifa': 13, 'Solo_Parang': 14, 'SulawesiSelatan_Lontara': 15, 'SumateraBarat_Rumah_Minang': 16, 'SumateraUtara_Boraspati': 17, 'Yogyakarta_Kawung': 18, 'Yogyakarta_Parang': 19}


### Dataset

In [ ]:
num_images = train_generator.n
num_classes = train_generator.num_classes
print(f"Number of images: {train_generator.n}")
print(f"Number of classes:{train_generator.num_classes}")

Number of images: 520
Number of classes:20


In [ ]:
x_train_all = []
y_train_all = []

for _ in range(num_images // batch_size + 1):
    x_batch, y_batch = next(train_generator)
    x_train_all.append(x_batch)
    y_train_all.append(y_batch)

x_train = np.concatenate(x_train_all, axis=0)
y_train = np.concatenate(y_train_all, axis=0)

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)

x_train shape: (520, 224, 224, 3)
y_train shape: (520, 20)


In [ ]:
val_num_images = validation_generator.n
val_num_classes = validation_generator.num_classes
print(f"Number of images: {validation_generator.n}")
print(f"Number of classes:{validation_generator.num_classes}")

Number of images: 120
Number of classes:20


In [ ]:
x_val_all = []
y_val_all = []

for _ in range(val_num_images // batch_size + 1):
    x_val_batch, y_val_batch = next(validation_generator)
    x_val_all.append(x_val_batch)
    y_val_all.append(y_val_batch)

x_val = np.concatenate(x_val_all, axis=0)
y_val = np.concatenate(y_val_all, axis=0)

In [ ]:
test_no_images = test_generator.n
test_no_classes = test_generator.num_classes
print(f"Number of images: {test_generator.n}")
print(f"Number of classes:{test_generator.num_classes}")

Number of images: 160
Number of classes:20


In [ ]:
x_test_all = []
y_test_all = []

for _ in range(test_num_images // batch_size + 1):
    x_test_batch, y_test_batch = next(test_generator)
    x_test_all.append(x_test_batch)
    y_test_all.append(y_test_batch)

x_test = np.concatenate(x_test_all, axis=0)
y_test = np.concatenate(y_test_all, axis=0)

## Reshaping

In [ ]:
x_train = x_train.reshape(x_train.shape[0], TOTAL_INPUTS)

In [ ]:
x_validation = x_validation.reshape(x_validation.shape[0], TOTAL_INPUTS)

In [ ]:
x_test = x_test.reshape(x_test.shape[0], TOTAL_INPUTS)

# Create Simple Modelling

In [ ]:
model = Sequential()

model.add(Dense(units=128, input_dim=total_inputs, activation="relu", name="dense1"))
model.add(Dropout(0.2, seed=42))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(64, activation='relu', name='dense2'))
model.add(BatchNormalization())
model.add(Dense(16, activation='relu', name='dense3'))
model.add(BatchNormalization())
model.add(Dense(20, activation='softmax', name='output_layer'))

model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense1 (Dense)              (None, 128)               19267712  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 dense2 (Dense)              (None, 64)                8256      
                                                                 
 batch_normalization_1 (Bat  (None, 64)                256       
 chNormalization)                                     

In [ ]:
EPOCHS = 200

checkpoint_callback = ModelCheckpoint(filepath='/content/saved_models/raw_model.h5', save_best_only=False)
callbacks = [checkpoint_callback]

history4 = model.fit(x_train, y_train,
                      batch_size=batch_size, verbose=0,
                      epochs=EPOCHS, callbacks=callbacks,
                      validation_data=(x_validation, y_val))

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.predict(x_validation)

4/4 [==============================] - 0s 45ms/step


array([[7.006e-01, 3.707e-04, 6.276e-03, ..., 1.691e-02, 4.297e-06,
        7.783e-05],
       [4.566e-02, 6.008e-03, 3.769e-02, ..., 2.130e-01, 1.273e-05,
        3.884e-05],
       [4.060e-04, 5.633e-06, 9.677e-01, ..., 2.534e-04, 9.412e-06,
        1.864e-06],
       ...,
       [1.530e-01, 1.762e-01, 7.961e-02, ..., 6.683e-02, 1.201e-05,
        9.226e-04],
       [7.448e-03, 8.141e-02, 4.240e-03, ..., 3.330e-01, 1.067e-05,
        1.749e-04],
       [3.331e-02, 1.723e-03, 1.063e-03, ..., 8.365e-01, 2.751e-06,
        1.832e-05]], dtype=float32)

In [ ]:
model.predict(x_validation).shape

4/4 [==============================] - 0s 31ms/step


(120, 20)

In [ ]:
model.metrics_names

['loss', 'accuracy']

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test_normalized, y_test)
print(f"Test loss : {test_loss:0.3} and the test accuracy: {test_accuracy:0.2%}")

6/6 [==============================] - 0s 33ms/step - loss: 8.1159 - accuracy: 0.1146
Test loss : 8.12 and the test accuracy: 11.46%
